# Финальная работа поSQL 

# Описание проекта

Анализ базы данных для формулировки ценностного предложения для нового продукта.

# Обзор данных

### Подключается к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [4]:
def get_sql(name):
    display(pd.io.sql.read_sql(name, con = engine))

### Посмотрим на первые строки таблиц

In [5]:
books = '''
SELECT
*
FROM books
LIMIT 5
'''

authors = '''
SELECT
*
FROM authors
LIMIT 5
'''

publishers = '''
SELECT
*
FROM publishers
LIMIT 5
'''

ratings = '''
SELECT
*
FROM ratings
LIMIT 5
'''

reviews = '''
SELECT
*
FROM reviews
LIMIT 5
'''

In [6]:
get_sql(books)
get_sql(authors)
get_sql(publishers)
get_sql(ratings)
get_sql(reviews)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблицы, табличечки. Данные выглядят хорошо.

# Задачи

### 1. Посчитаем, сколько книг вышло после 1 января 2000 года

In [7]:
task1 = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date >= '2000-01-01'
'''

In [8]:
get_sql(task1)

,count
0,821


После 1 января 2000 года вышла 821 книга. За 20 лет можно все их прочитать,  хорошо, что в приложении есть и книги прошлой эпохи.

### 2. Для каждой книги посчитаем количество обзоров и среднюю оценку

In [9]:
task2 = '''
SELECT b.title,
       COUNT(DISTINCT review_id),
       AVG(rating)
FROM books AS b 
INNER JOIN ratings AS rt ON rt.book_id=b.book_id
LEFT OUTER JOIN reviews AS rv ON rv.book_id=b.book_id
GROUP BY b.book_id
'''

In [10]:
get_sql(task2)

,title,count,avg
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,Xenocide (Ender's Saga #3),3,3.400000
997,Year of Wonders,4,3.200000
998,You Suck (A Love Story #2),2,4.500000


Теперь у нас есть таблица с обзорами и оценками книг. Можно сортировать её во все стороны и получать рекомендации, продвижения и пр.

### 3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц 
так мы исключим из анализа брошюры

In [11]:
task3 = '''
SELECT publisher,
       COUNT(DISTINCT book_id)
FROM books AS b LEFT OUTER JOIN publishers AS p ON b.publisher_id=p.publisher_id 
WHERE num_pages > 50
GROUP BY publisher
ORDER BY count DESC
LIMIT 1
'''

In [12]:
get_sql(task3)

,publisher,count
0,Penguin Books,42


Penguin Books выпустило больше всего книг. С популярными издательствами стоит сотрудничать в первую очередь.

### 4. Определим автора с самой высокой средней оценкой книг
учитывайте только книги с 50 и более оценками

In [13]:
task4 = '''
SELECT a.author,
       b.title,
       AVG(rating)
FROM authors AS a 
JOIN books AS b ON a.author_id=b.author_id
JOIN ratings AS r ON b.book_id=r.book_id
GROUP BY a.author, b.title
HAVING COUNT(DISTINCT rating_id)>50
ORDER BY avg DESC
LIMIT 1
'''

In [14]:
get_sql(task4)

,author,title,avg
0,J.K. Rowling/Mary GrandPré,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634


Читатели высоко ценят книгиДж.К. Роулинг за Гарри Поттера и иллюстрации к ним от Мэри Гранпрэ

### 5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [15]:
task5 = '''
WITH i AS (
SELECT username,
       COUNT(review_id)
FROM reviews
WHERE reviews.username in (
SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(DISTINCT rating_id)>50 
)
GROUP BY username
)
SELECT AVG(count)
FROM i
'''

In [16]:
get_sql(task5)

,avg
0,24.333333


В среднем, наиболее активные пользователи пишут по 24 обзора на книги. Это довольно много. Может, книголюбы и сами не прочь быть авторами?

# Выводы

В данных можно найти информацию о современных книгах: 821 книга вышла в этом тысячелетии. Можно использовать данные о ее рейтинге и количестве обзоров для вывода рекомендаций или продвижения в топ книг. 

Также есть информация и об издательствах, например Penguin Books, наиболее активное из представленных, выпустило 43 книги. Есть возможность приобретать в будущем книги из популярных издательств.

Имея данные по средней оценке, можно выбирать авторов с высокой оценкой для выставленияих книг в приложение.

База данных содержит информацию о пользовательских оценках и обзорах. Пользователи довольно активны, стоит продолжать работу с обозами, возможно, улучшая эту функцию или предоставляя бонусы  наиболее активным.